Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [38]:
from itertools import product
from functools import reduce
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse
from copy import copy

In [39]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True

    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [64]:
NUM_POINTS = 100

In [65]:
def evaluate(state):
    cost = sum(state)
    
    valid = np.all(
        reduce(
            np.logical_or,
            [x.getrow(i).toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost if valid else 0

In [66]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, NUM_POINTS - 1)
    new_state[index] = not new_state[index]
    return new_state

In [71]:
current_state = [choice([True, False]) for _ in range(NUM_POINTS)]
best_state = [False for _ in range(NUM_POINTS)]

print(current_state, evaluate(current_state))

for _ in range(100):
    x = make_set_covering_problem(NUM_POINTS, NUM_POINTS, .7)
    for step in range(100):
        new_state = tweak(current_state)
        if evaluate(new_state) >= evaluate(current_state):
            current_state = new_state
            #print(current_state, evaluate(current_state))
    if evaluate(current_state) >= evaluate(best_state):
        best_state = current_state


[True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, False, False, True, True, True, False, True, True, True, True, False, True, False, True, False, True, False, True, False, False, True, False, False, True, False, True, True, False, True, False, True, True, False, True, True, False, True, True, True, True, True, False, True, False, False, False, True, True, True, False, False, False, False, True, True, True, True, False, True, False, False, False, False, True, True, True, True, True, True, True, True, True, False, False, True, True, True, True, False, True, False, False, True, False, False] (True, -63)


In [70]:
print(f"{NUM_POINTS}, {evaluate(current_state)}")

100, (True, -29)
